# Build an Agent



> LangChain supports the creation of agents, or systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.



In [1]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.9 MB/s eta 0:00:00


In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()


··········


In [4]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in Jalandhar")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in Jalandhar', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Jalandhar', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Jalandhar', 'region': 'Punjab', 'country': 'India', 'lat': 31.3256, 'lon': 75.5792, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1754869541, 'localtime': '2025-08-11 05:15'}, 'current': {'last_updated_epoch': 1754869500, 'last_updated': '2025-08-11 05:15', 'temp_c': 27.5, 'temp_f': 81.5, 'is_day': 0, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.7, 'wind_kph': 7.6, 'wind_degree': 145, 'wind_dir': 'SE', 'pressure_mb': 1002.0, 'pressure_in': 29.58, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 81, 'cloud': 45, 'feelslike_c': 31.4, 'feelslike_f': 88.4, 'windchill_c': 27.5, 'windchill_f': 81.5, 'heatindex_c': 31.4, 'heatindex_f': 88.4, 'dewpoint_c': 23.9, 'dewpoint_f': 75.0, 'vis_km': 10.

In [5]:
!pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini: ··········


In [7]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

In [10]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.



In [11]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [12]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in SF'}, 'id': '45b4eaa8-9cd8-490e-9b58-4b98df094358', 'type': 'tool_call'}]


This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [14]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [15]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (9d85f8b0-6c24-4981-81e3-eae128f72543)
 Call ID: 9d85f8b0-6c24-4981-81e3-eae128f72543
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1754870370, 'localtime': '2025-08-10 16:59'}, 'current': {'last_updated_epoch': 1754869500, 'last_updated': '2025-08-10 16:45', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 1, 'condition': {

In [16]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (bcdcccaa-4608-4406-8b28-bbd3395e6942)
 Call ID: bcdcccaa-4608-4406-8b28-bbd3395e6942
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1754870370, 'localtime': '2025-08-10 16:59'}, 'current': {'last_updated_epoch': 1754869500, 'last_updated': '2025-08-10 16:45', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 1, 'condition': {

In [17]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in San Francisco, California on August 10, 2025, at 4:45 PM is partly cloudy with a temperature of 66.9°F (19.4°C). The wind is| from the WSW at 12.1 mph (19.4 kph) with gusts up to 19.8 mph (31.9 kph). Humidity is 71%, and there is no precipitation.| The pressure is 29.95 in (1014.0 mb).|

**Adding in memory**

In [18]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [19]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [20]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! How can I help you today?


In [21]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I'm sorry, I don't have access to past conversations and cannot recall your name.
